In [2]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import urllib.request
import urllib.parse
import pandas as pd
import time
#----------스크롤 코드------------------
driver = webdriver.Chrome('./chromedriver.exe')
url = 'https://www.youtube.com/channel/UCNFkIqiyp6wcLKNv2o8H8eQ/videos'
driver.get(url)
SCROLL_PAUSE_TIME=0.5
body = driver.find_element_by_tag_name('body')
while True:
    last_height=driver.execute_script('return document.documentElement.scrollHeight')
    for i in range(10):
        body.send_keys(Keys.END)
        time.sleep(SCROLL_PAUSE_TIME)
    new_height= driver.execute_script('return document.documentElement.scrollHeight')
    if new_height == last_height:
        break
soup = bs(driver.page_source, 'html.parser')
video_url = soup.select('a#video-title')
url_list = []
for i in video_url:
    url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))
source=[]
for i in range(len(url_list)):
    index=url_list[i].index('=')
    source.append(url_list[i][index+1:len(url_list[i])])
driver.close()

In [3]:
data=[]
result=[]
Num=0
while Num < len(source):
    try:
        for i in range(len(source)):
            driver = webdriver.Chrome('./chromedriver.exe')
            url = 'https://www.youtube.com/watch?v={}'.format(source[i])
            driver.get(url)
            SCROLL_PAUSE_TIME=0.5
            body = driver.find_element_by_tag_name('body')
            while True:
                last_height=driver.execute_script('return document.documentElement.scrollHeight')
                for i in range(10):
                    body.send_keys(Keys.END)
                    time.sleep(SCROLL_PAUSE_TIME)
                new_height= driver.execute_script('return document.documentElement.scrollHeight')
                if new_height == last_height:
                    break
            soup = bs(driver.page_source, 'html.parser')
            title=soup.select('.title.style-scope.ytd-video-primary-info-renderer')[0].get_text()
            view=soup.select('.view-count.style-scope.yt-view-count-renderer')[0].get_text()
            date=soup.select('div #date yt-formatted-string.style-scope.ytd-video-primary-info-renderer')[0].get_text()
            content=soup.select('.content.style-scope.ytd-video-secondary-info-renderer')[0].get_text()
            like=soup.select('.style-scope.ytd-toggle-button-renderer.style-text')[1].get_text()
            unlike=soup.select('.style-scope.ytd-toggle-button-renderer.style-text')[3].get_text()
            soupstr=str(soup)
            if 'count-text style-scope ytd-comments-header-renderer' in soupstr:
                comment=soup.select('.count-text.style-scope.ytd-comments-header-renderer')[0].get_text()
            else:
                comment=0
            data=[title,view,date,content,like,unlike,comment]
            result.append(data)
            driver.close()
            Num+=1
    except:
        break

In [4]:
df=pd.DataFrame(result)
df.columns=['제목','조회 수','게시일','내용','좋아요','싫어요','댓글 수']
df

,제목,조회 수,게시일,내용,좋아요,싫어요,댓글 수
0,Have fun in the Top French Cities,조회수 393회,2019. 10. 16.,The Top French Cities are an association of 29...,1,1,댓글 0개
1,What's happening in France in 2020,조회수 375회,2019. 10. 16.,"All the tourism and cultural news, including n...",4,0,댓글 0개
2,"Exceptional exhibition Rockwell, Roosevelt T...",조회수 107회,2019. 5. 22.,"Enduring ideals: Rockwell, Roosevelt & the Fou...",0,0,댓글 0개
3,Normandy - 75th anniversary of the D-Day Landings,조회수 291회,2019. 4. 26.,See the major developments for the 75th annive...,3,0,댓글 0개
4,Loire Valley - 500th anniversary of the French...,조회수 81회,2019. 4. 26.,This anniversary coincides with the 500th anni...,1,0,댓글 0개
...,...,...,...,...,...,...,...
70,Les Portes du Soleil,조회수 35회,2012. 9. 28.,Morzine & Les Gets are two must-see resorts in...,0,0,댓글 0개
71,Paradiski,조회수 69회,2012. 9. 28.,"Situated in the heart of Savoy, Paradiski is a...",0,0,댓글 0개
72,Espace Killy - Tignes - X Games Best Of 2012,조회수 35회,2012. 9. 28.,"Tignes is for the sporty, freestylers, cosmopo...",0,0,댓글 0개
73,Les 3 Vallees,조회수 469회,2012. 9. 28.,The largest ski area in the world includes Cou...,4,0,댓글 0개


In [5]:
df.to_excel('Youtube_Crawling.xlsx',encoding='CP949')